In [ ]:
import pandas as pd
import numpy as np
import re
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from pathlib import Path
import os

In [ ]:
DATA = Path(".")  # folder where your CSVs live
OUT  = Path("./acs_clean"); OUT.mkdir(exist_ok=True)

FILES = {
    "population": DATA/"USCPOPDATA.csv",
    "housing": DATA/"USCHOUSINGDATA.csv",
    "tenure": DATA/"USCHOUSETENUREDATA.csv",
    "commute_time": DATA/"USCCOMMUTETIMEDATA.csv",
    "transport_type": DATA/"USCTRANSPORTTYPEDATA.csv",
    "vehicles": DATA/"USCVEHICLESAVAILABLEDATA.csv",
    "sex_by_age": DATA/"USCSEXBYAGEDATA.csv",
}

In [ ]:
def _first_estimate_column(df):
    # Find the Estimate column like "Davidson County, Tennessee!!Estimate"
    for c in df.columns:
        if isinstance(c, str) and c.endswith("!!Estimate"):
            return c
    raise ValueError("Estimate column not found (expected to end with '!!Estimate').")

In [ ]:
def _first_moe_column(df):
    # Find the Margin of Error column like "Davidson County, Tennessee!!Margin of Error"
    for c in df.columns:
        if isinstance(c, str) and c.endswith("!!Margin of Error"):
            return c
    return None  # some downloads may lack MOE

In [ ]:
def _to_number(x):
    # Strip commas, ±, spaces; coerce to numeric
    if pd.isna(x):
        return np.nan
    return pd.to_numeric(str(x).replace(",", "").replace("±","").strip(), errors="coerce")

In [ ]:
def clean_acs_single_table(path, table_name):
    df = pd.read_csv(path)
    cat_col = "Label (Grouping)" if "Label (Grouping)" in df.columns else "Label"
    est_col = _first_estimate_column(df)
    moe_col = _first_moe_column(df)

    keep = [cat_col, est_col] + ([moe_col] if moe_col else [])
    out = df[keep].copy()

    out = out.rename(columns={
        cat_col: "category",
        est_col: "estimate",
        (moe_col or "moe"): "moe"
    })

    out["estimate"] = out["estimate"].apply(_to_number)
    if moe_col:
        out["moe"] = out["moe"].apply(_to_number)
    else:
        out["moe"] = np.nan

    # trim whitespace and normalize the category text
    out["category"] = out["category"].astype(str).str.replace(r"\u00a0", " ", regex=True).str.strip()
    out.insert(0, "table", table_name)       # add table name
    out.insert(1, "geo", "Davidson County, TN")
    return out

In [ ]:
cleaned = []
for name, path in FILES.items():
    cleaned.append(clean_acs_single_table(path, name))
acs_long = pd.concat(cleaned, ignore_index=True)

acs_long.to_csv(OUT/"acs_davidson_2023_long.csv", index=False)
acs_long.head(10)

In [ ]:
acs_long.groupby("table").size().rename("rows")

In [ ]:
for tbl in FILES.keys():
    print("\n===", tbl, "===")
    display(acs_long.loc[acs_long.table.eq(tbl), ["category","estimate","moe"]].head(8))


In [ ]:
def _norm_cat(s: str) -> str:
    if pd.isna(s): return ""
    s = str(s).replace("\u00a0"," ")
    s = re.sub(r"\s+"," ", s).strip().rstrip(":")
    return s.lower()

def _pick_any(df: pd.DataFrame, patterns):
    """Return the first matching estimate for any pattern (exact OR prefix if pattern ends with ':')."""
    if isinstance(patterns, str):
        patterns = [patterns]
    if df.empty:
        return None
    cats = df["category"].map(_norm_cat)
    for pat in patterns:
        pat_n = _norm_cat(pat)
        allow_prefix = pat.strip().endswith(":")
        mask = cats.str.startswith(pat_n) if allow_prefix else (cats == pat_n)
        vals = pd.to_numeric(df.loc[mask, "estimate"], errors="coerce")
        if not vals.empty:
            v = vals.max()
            if pd.notna(v):
                return float(v)
    return None

def summarize_tables(acs_long: pd.DataFrame) -> pd.DataFrame:
    out = {}

    # Population
    pop = acs_long.query("table == 'population'")
    out["population_total"] = _pick_any(pop, ["total:", "total population"])

    # Housing units
    housing = acs_long.query("table == 'housing'")
    out["housing_units_total"] = _pick_any(housing, ["total housing units", "total:"])

    # Tenure
    tenure = acs_long.query("table == 'tenure'")
    out["occupied_total"]  = _pick_any(tenure, ["total:"])
    out["owner_occupied"]  = _pick_any(tenure, ["owner occupied", "occupied housing units:"])
    out["renter_occupied"] = _pick_any(tenure, ["renter occupied", "occupied housing units: renter occupied"])

    # Vehicles available
    veh = acs_long.query("table == 'vehicles'")
    out["hh_total_for_veh"] = _pick_any(veh, ["total:"])
    out["hh_0_veh"]         = _pick_any(veh, ["no vehicle available", "no vehicles available"])
    out["hh_1_veh"]         = _pick_any(veh, ["1 vehicle available"])
    out["hh_2_veh"]         = _pick_any(veh, ["2 vehicles available"])
    out["hh_3plus_veh"]     = _pick_any(veh, ["3 or more vehicles available", "3 vehicles available"])

    # Commute time distribution
    ct = acs_long.query("table == 'commute_time'")
    out["workers_total"]    = _pick_any(ct, ["total:"])
    out["gt_30min_commute"] = sum([
        _pick_any(ct, "30 to 34 minutes") or 0,
        _pick_any(ct, "35 to 39 minutes") or 0,
        _pick_any(ct, "40 to 44 minutes") or 0,
        _pick_any(ct, "45 to 59 minutes") or 0,
        _pick_any(ct, "60 to 89 minutes") or 0,
        _pick_any(ct, "90 or more minutes") or 0,
    ])

    # Transport type / mode share
    mode = acs_long.query("table == 'transport_type'")
    out["mode_total"]         = _pick_any(mode, ["total:"])
    out["drive_alone"]        = _pick_any(mode, ["drove alone"])  
    out["carpool"]            = _pick_any(mode, ["carpooled:"]) 
    out["public_transport"]   = _pick_any(mode, ["public transportation (excluding taxicab)"])
    out["walked"]             = _pick_any(mode, ["walked"])
    out["bicycle"]            = _pick_any(mode, ["bicycle"])
    out["worked_from_home"]   = _pick_any(mode, ["worked from home"])

    return pd.DataFrame([out])

In [ ]:
acs_wide = summarize_tables(acs_long)
print(type(acs_wide), acs_wide.shape)
display(acs_wide.T)

In [ ]:
w = acs_wide.iloc[0]

acs_metrics = pd.Series({
    # mode shares
    "drive_alone_share":        w["drive_alone"] / w["mode_total"],
    "carpool_share":            w["carpool"] / w["mode_total"],
    "public_transport_share":   w["public_transport"] / w["mode_total"],
    "walk_share":               w["walked"] / w["mode_total"],
    "bike_share":               w["bicycle"] / w["mode_total"],
    "wfh_share":                w["worked_from_home"] / w["mode_total"],

    # vehicles per household shares
    "zero_vehicle_share":       w["hh_0_veh"] / w["hh_total_for_veh"],
    "one_vehicle_share":        w["hh_1_veh"] / w["hh_total_for_veh"],
    "two_vehicle_share":        w["hh_2_veh"] / w["hh_total_for_veh"],
    "three_plus_vehicle_share": w["hh_3plus_veh"] / w["hh_total_for_veh"],

    # tenure shares
    "owner_share":              w["owner_occupied"] / w["occupied_total"],
    "renter_share":             w["renter_occupied"] / w["occupied_total"],
})

acs_metrics.to_frame("value").to_csv(OUT/"acs_davidson_2023_metrics.csv")
acs_metrics.to_frame("value").round(3)

In [ ]:
print("mode shares sum:", float(acs_metrics[[
    "drive_alone_share","carpool_share","public_transport_share","walk_share","bike_share","wfh_share"
]].sum()))

print("vehicle shares sum:", float(acs_metrics[[
    "zero_vehicle_share","one_vehicle_share","two_vehicle_share","three_plus_vehicle_share"
]].sum()))

print("tenure shares sum:", float(acs_metrics[["owner_share","renter_share"]].sum()))

In [ ]:
ms = acs_metrics[[
    "drive_alone_share","carpool_share","public_transport_share",
    "walk_share","bike_share","wfh_share"
]].sort_values(ascending=False)

plt.figure(figsize=(8,4))
plt.bar(ms.index.str.replace("_share","").str.replace("_"," ").str.title(), ms.values)
plt.title("Mode Shares — Davidson County (ACS 5-Year 2019–2023)")
plt.ylabel("Share of commuters")
plt.xticks(rotation=20)
plt.tight_layout()
plt.show()

In [ ]:
out_all = pd.concat([acs_wide.T.rename(columns={0:"value"}), acs_metrics.to_frame("value")])
out_all.to_csv(OUT/"acs_davidson_2023_summary_plus_metrics.csv")
out_all
print(out_all.reset_index().rename(columns={"index":"metric"}).head(15))

In [ ]:
cats = (acs_long.loc[acs_long.table.eq("transport_type"), "category"]
        .dropna().drop_duplicates().sort_values())
cats.to_list()

In [ ]:
print(type(acs_long))


In [ ]:
print(len(acs_long))

In [ ]:
for tbl in ["population","housing","tenure","vehicles","commute_time","transport_type"]:
    print("\n=== ", tbl, " ===")
    cats = (acs_long
            .loc[acs_long.table.eq(tbl), "category"]
            .dropna()
            .drop_duplicates()
            .sort_values())
    display(cats.head(25))

In [ ]:
w = acs_wide.iloc[0]

acs_metrics = pd.Series({
    "owner_share": w["owner_occupied"] / w["occupied_total"] if w["occupied_total"] else np.nan,
    "renter_share": w["renter_occupied"] / w["occupied_total"] if w["occupied_total"] else np.nan,
    "zero_vehicle_share": w["hh_0_veh"] / w["hh_total_for_veh"] if w["hh_total_for_veh"] else np.nan,
    "one_vehicle_share":  w["hh_1_veh"] / w["hh_total_for_veh"] if w["hh_total_for_veh"] else np.nan,
    "two_vehicle_share":  w["hh_2_veh"] / w["hh_total_for_veh"] if w["hh_total_for_veh"] else np.nan,
    "three_plus_vehicle_share": w["hh_3plus_veh"] / w["hh_total_for_veh"] if w["hh_total_for_veh"] else np.nan,
    "gt_30min_commute_share": w["gt_30min_commute"] / w["workers_total"] if w["workers_total"] else np.nan,
    "drive_alone_share":  w["drive_alone"] / w["mode_total"] if w["mode_total"] else np.nan,
    "carpool_share":      w["carpool"] / w["mode_total"] if w["mode_total"] else np.nan,
    "public_transport_share": w["public_transport"] / w["mode_total"] if w["mode_total"] else np.nan,
    "walk_share":         w["walked"] / w["mode_total"] if w["mode_total"] else np.nan,
    "bike_share":         w["bicycle"] / w["mode_total"] if w["mode_total"] else np.nan,
    "wfh_share":          w["worked_from_home"] / w["mode_total"] if w["mode_total"] else np.nan,
})

acs_metrics.to_frame("value").to_csv(OUT/"acs_davidson_2023_metrics.csv")
acs_metrics.to_frame("value")


In [ ]:
for tbl in FILES.keys():
    acs_long.loc[acs_long.table.eq(tbl), :].to_csv(OUT/f"acs_{tbl}_2023_long.csv", index=False)
print("Wrote individual cleaned tables to:", OUT)

In [ ]:
CANDIDATES = {
    "income":       ["USCINCOMEDATA.csv", "USCMEDHHINCOME.csv"],
    "poverty":      ["USCPOVERTYDATA.csv"],
    "employment":   ["USCEMPLOYMENTDATA.csv"],
    "education":    ["USCEDUCATIONDATA.csv"],
    "rent":         ["USCMEDRENT.csv", "USCMEDRENTDATA.csv", "USCRENTPAYDATA.csv"],
    "home_value":   ["USCHOMEVALUEDATA.csv", "USCMEDHOMEVAL.csv"],
}

def find_file(names, search_dir=Path(".")):
    for name in names:
        p = search_dir / name
        if p.exists():
            return p
    return None

NEW_FILES = {k: find_file(v) for k, v in CANDIDATES.items()}
FOUND = {k: v for k, v in NEW_FILES.items() if v is not None}
MISSING = [k for k, v in NEW_FILES.items() if v is None]

print("Found:", FOUND)
print("Missing:", MISSING or "None")

# ❷ Merge into your existing FILES dict only for ones we actually found
try:
    FILES.update(FOUND)
except NameError:
    FILES = FOUND.copy()

In [ ]:
new_long = []
for name in ["income","poverty","employment","education","rent","home_value"]:
    if name in FILES:
        new_long.append(clean_acs_single_table(FILES[name], name))

if new_long:
    new_long = pd.concat(new_long, ignore_index=True)
    try:
        acs_long = pd.concat([acs_long, new_long], ignore_index=True)
    except NameError:
        acs_long = new_long.copy()
else:
    print("No new ACS tables were found on disk to append.")

# optional: save the refreshed long file alongside your existing outputs
OUT.mkdir(exist_ok=True)
(OUT / "acs_davidson_long_latest.csv").write_text(acs_long.to_csv(index=False))
print("acs_long rows:", len(acs_long))

In [ ]:
def summarize_tables_extended(acs_long: pd.DataFrame) -> pd.DataFrame:
    out = {}

    # --- already in your workbook ---
    pop   = acs_long.query("table == 'population'")
    hous  = acs_long.query("table == 'housing'")
    ten   = acs_long.query("table == 'tenure'")
    veh   = acs_long.query("table == 'vehicles'")
    ct    = acs_long.query("table == 'commute_time'")
    mode  = acs_long.query("table == 'transport_type'")

    out["population_total"]      = _pick_any(pop, ["total:", "total population"])
    out["housing_units_total"]   = _pick_any(hous, ["total housing units", "total:"])
    out["occupied_total"]        = _pick_any(ten,  ["total:"])
    out["owner_occupied"]        = _pick_any(ten,  ["owner occupied", "occupied housing units:"])
    out["renter_occupied"]       = _pick_any(ten,  ["renter occupied", "occupied housing units: renter occupied"])
    out["hh_total_for_veh"]      = _pick_any(veh,  ["total:"])
    out["hh_0_veh"]              = _pick_any(veh,  ["no vehicle available", "no vehicles available"])
    out["hh_1_veh"]              = _pick_any(veh,  ["1 vehicle available"])
    out["hh_2_veh"]              = _pick_any(veh,  ["2 vehicles available"])
    out["hh_3plus_veh"]          = _pick_any(veh,  ["3 or more vehicles available", "3 vehicles available"])
    out["workers_total"]         = _pick_any(ct,   ["total:"])
    out["gt_30min_commute"]      = sum([
        _pick_any(ct, "30 to 34 minutes") or 0,
        _pick_any(ct, "35 to 39 minutes") or 0,
        _pick_any(ct, "40 to 44 minutes") or 0,
        _pick_any(ct, "45 to 59 minutes") or 0,
        _pick_any(ct, "60 to 89 minutes") or 0,
        _pick_any(ct, "90 or more minutes") or 0,
    ])
    out["mode_total"]            = _pick_any(mode, ["total:"])
    out["drive_alone"]           = _pick_any(mode, ["drove alone"])
    out["carpool"]               = _pick_any(mode, ["carpooled:"])  # header with colon matches your labels
    out["public_transport"]      = _pick_any(mode, ["public transportation (excluding taxicab):"])
    out["walked"]                = _pick_any(mode, ["walked"])
    out["bicycle"]               = _pick_any(mode, ["bicycle"])
    out["worked_from_home"]      = _pick_any(mode, ["worked from home"])

    # --- NEW topics (only if present) ---
    # Income (B19013)
    inc = acs_long.query("table == 'income'")
    if not inc.empty:
        out["median_household_income"] = _pick_any(inc, [
            "median household income in the past 12 months (in 2023 inflation-adjusted dollars)",
            "median household income in the past 12 months (in inflation-adjusted dollars)",
            "median household income in the past 12 months (in 20",  # fallback prefix
        ])

    # Poverty (B17001)
    pov = acs_long.query("table == 'poverty'")
    if not pov.empty:
        out["poverty_denominator"] = _pick_any(pov, ["total:"])
        out["below_poverty_total"] = _pick_any(pov, ["below poverty level:"])

    # Employment (B23025)
    emp = acs_long.query("table == 'employment'")
    if not emp.empty:
        out["pop_16plus"]       = _pick_any(emp, ["civilian noninstitutional population"])
        out["in_labor_force"]   = _pick_any(emp, ["in labor force:"])
        out["civilian_lf"]      = _pick_any(emp, ["civilian labor force:"])
        out["employed"]         = _pick_any(emp, ["employed"])
        out["unemployed"]       = _pick_any(emp, ["unemployed"])
        out["not_in_labor"]     = _pick_any(emp, ["not in labor force"])

    # Education (B15003, 25+)
    edu = acs_long.query("table == 'education'")
    if not edu.empty:
        out["edu_total_25plus"] = _pick_any(edu, ["total:"])
        hs_plus = sum([
            _pick_any(edu, "regular high school diploma") or 0,
            _pick_any(edu, "ged or alternative credential") or 0,
            _pick_any(edu, "some college, less than 1 year") or 0,
            _pick_any(edu, "some college, 1 or more years, no degree") or 0,
            _pick_any(edu, "associate's degree") or 0,
            _pick_any(edu, "bachelor's degree") or 0,
            _pick_any(edu, "master's degree") or 0,
            _pick_any(edu, "professional school degree") or 0,
            _pick_any(edu, "doctorate degree") or 0,
        ])
        bach_plus = sum([
            _pick_any(edu, "bachelor's degree") or 0,
            _pick_any(edu, "master's degree") or 0,
            _pick_any(edu, "professional school degree") or 0,
            _pick_any(edu, "doctorate degree") or 0,
        ])
        grad_plus = sum([
            _pick_any(edu, "master's degree") or 0,
            _pick_any(edu, "professional school degree") or 0,
            _pick_any(edu, "doctorate degree") or 0,
        ])
        out["edu_hs_or_higher"]     = hs_plus
        out["edu_bachelors_plus"]   = bach_plus
        out["edu_grad_prof_plus"]   = grad_plus

    # Housing costs
    rent = acs_long.query("table == 'rent'")
    if not rent.empty:
        out["median_gross_rent"] = _pick_any(rent, ["median gross rent (dollars)", "median gross rent"])
    hv = acs_long.query("table == 'home_value'")
    if not hv.empty:
        out["median_home_value"] = _pick_any(hv, ["median value (dollars)", "median value"])

    return pd.DataFrame([out])

In [ ]:
acs_wide_ext = summarize_tables_extended(acs_long)
display(acs_wide_ext.T)

w = acs_wide_ext.iloc[0]
derived = pd.Series({
    "unemployment_rate":   (w["unemployed"] / w["civilian_lf"]) if w.get("civilian_lf") else None,
    "labor_participation": (w["in_labor_force"] / w["pop_16plus"]) if w.get("pop_16plus") else None,
    "poverty_rate":        (w["below_poverty_total"] / w["poverty_denominator"]) if w.get("poverty_denominator") else None,
    "hs_or_higher_share":  (w["edu_hs_or_higher"] / w["edu_total_25plus"]) if w.get("edu_total_25plus") else None,
    "bachelors_plus_share":(w["edu_bachelors_plus"] / w["edu_total_25plus"]) if w.get("edu_total_25plus") else None,
    "grad_prof_plus_share":(w["edu_grad_prof_plus"] / w["edu_total_25plus"]) if w.get("edu_total_25plus") else None,
})

out_all_ext = pd.concat([acs_wide_ext.T.rename(columns={0:"value"}), derived.to_frame("value")])
out_path = OUT / "acs_davidson_summary_plus_metrics_extended.csv"
out_all_ext.to_csv(out_path)
print("Wrote:", out_path)
print(out_all_ext.reset_index().rename(columns={"index":"metric"}).head(20))

In [ ]:
for tbl in ["income","poverty","employment","education","rent","home_value"]:
    if tbl in FILES:
        print("\n==", tbl.upper(), "==", FILES[tbl])
        cats = (acs_long.loc[acs_long.table.eq(tbl), "category"]
                .dropna().drop_duplicates().sort_values())
        for c in cats[:200]:
            print(c)

In [ ]:
for k in ["income","poverty","employment","education","rent","home_value"]:
    print(f"{k:12s} →", FILES.get(k))

In [ ]:
ACS_2023 = {
    "population":      Path("USCPOPDATA.csv"),
    "housing":         Path("USCHOUSINGDATA.csv"),
    "tenure":          Path("USCHOUSETENUREDATA.csv"),
    "vehicles":        Path("USCVEHICLESAVAILABLEDATA.csv"),
    "commute_time":    Path("USCCOMMUTETIMEDATA.csv"),
    "transport_type":  Path("USCTRANSPORTTYPEDATA.csv"),
    "sex_by_age":      Path("USCSEXBYAGEDATA.csv"),
    # add new topics here when you have them (same base name pattern):
    # "income":        Path("USCINCOMEDATA.csv"),
    # "poverty":       Path("USCPOVERTYDATA.csv"),
    # "employment":    Path("USCEMPLOYMENTDATA.csv"),
    # "education":     Path("USCEDUCATIONDATA.csv"),
    # "rent":          Path("USCRENTPAYDATA.csv"),
    # "home_value":    Path("USCHOMEVALUEDATA.csv"),
}

def with_year(p: Path, year: int) -> Path:
    """Insert the year just before the .csv extension: name.csv -> nameYYYY.csv"""
    if p.suffix.lower() != ".csv":
        return p  # safety: only handle csvs
    return p.with_name(p.stem + str(year) + p.suffix)

YEARS = [2023, 2018, 2013]

In [ ]:
files_by_year = {
    topic: {y: (with_year(base, y) if y != 2023 else base) for y in YEARS}
    for topic, base in ACS_2023.items()
}

In [ ]:
rows = []
for topic, by_year in files_by_year.items():
    for y, path in by_year.items():
        rows.append({"topic": topic, "year": y, "path": str(path), "exists": path.exists()})
availability = pd.DataFrame(rows).sort_values(["topic","year"])
print(availability)

In [ ]:
def preview_topic_year(topic: str, year: int, n: int = 5):
    """Load and show the top rows of a topic/year file."""
    path = files_by_year[topic][year]
    df = pd.read_csv(path)
    print(f"\n--- {topic.upper()} {year} ---")
    print(df.shape)
    display(df.head(n))
    return df

# Example: preview transport_type for 2018
df_transport_2018 = preview_topic_year("transport_type", 2018)

In [ ]:
preview_topic_year("vehicles", 2013)
preview_topic_year("housing", 2023)
preview_topic_year("sex_by_age", 2018)

In [ ]:
# Define expected columns for each topic
standard_columns = {
    "population": ["population_total"],
    "housing": ["housing_units_total", "occupied_total"],
    "tenure": ["owner_occupied", "renter_occupied"],
    "vehicles": ["hh_total_for_veh", "hh_0_veh", "hh_1_veh", "hh_2_veh", "hh_3plus_veh"],
    "commute_time": ["workers_total", "gt_30min_commute"],
    "transport_type": ["mode_total", "drive_alone", "carpool", "public_transport", "walked", "bicycle", "worked_from_home"],
    "sex_by_age": ["male_total", "female_total", "median_age"]
}

In [ ]:
def clean_standardize(topic: str, year: int) -> pd.DataFrame:
    """Load, clean, and standardize ACS data for a given topic/year."""
    path = files_by_year[topic][year]
    df = pd.read_csv(path)

    # Select + rename
    keep_cols = standard_columns[topic]
    df = df.reindex(columns=keep_cols)

    # Add topic/year context
    df["topic"] = topic
    df["year"] = year
    
    return df

In [ ]:
def combine_topic(topic: str, years=[2013, 2018, 2023]):
    frames = [clean_standardize(topic, y) for y in years]
    return pd.concat(frames, ignore_index=True)

# Example: Combine vehicle availability across years
vehicles_all = combine_topic("vehicles")
display(vehicles_all)

In [ ]:
cleaned_data = {
    topic: combine_topic(topic)
    for topic in standard_columns.keys()
}

In [ ]:
def check_columns(topic: str, years=[2013, 2018, 2023]):
    expected = set(standard_columns[topic])
    
    for year in years:
        path = files_by_year[topic][year]
        df = pd.read_csv(path)
        actual = set(df.columns)
        
        missing = expected - actual
        extra   = actual - expected
        
        print(f"\n{topic.upper()} — {year}")
        print(f"  Missing: {missing if missing else 'None'}")
        print(f"  Extra:   {extra if extra else 'None'}")

In [ ]:
for topic in standard_columns.keys():
    check_columns(topic)

In [ ]:
def _find_est_col(df: pd.DataFrame) -> str:
    """Return the column name ending with '!!Estimate' (as in ACS downloads)."""
    for c in df.columns:
        if isinstance(c, str) and c.endswith("!!Estimate"):
            return c
    raise ValueError("No '*!!Estimate' column found.")

def _norm_label(s: str) -> str:
    """Normalize category labels to improve matching."""
    if pd.isna(s): return ""
    s = str(s).replace("\u00a0", " ")              # nbsp
    s = re.sub(r"\s+", " ", s).strip()             # collapse spaces
    s = s.rstrip(":")                              # drop trailing colon
    return s.lower()

def _pick_any(df_cat: pd.DataFrame, patterns) -> float | None:
    """Pick estimate by matching normalized label against any pattern.
       If a pattern ends with ':' we allow prefix matching (hierarchy headers)."""
    if isinstance(patterns, str):
        patterns = [patterns]
    if df_cat.empty:
        return None
    labels_n = df_cat["category_n"]
    for pat in patterns:
        pat_n = _norm_label(pat)
        allow_prefix = pat.strip().endswith(":")
        mask = labels_n.str.startswith(pat_n) if allow_prefix else (labels_n == pat_n)
        vals = pd.to_numeric(df_cat.loc[mask, "estimate"], errors="coerce")
        if not vals.empty:
            v = vals.max()
            if pd.notna(v):
                return float(v)
    return None

def _load_raw(topic: str, year: int) -> pd.DataFrame:
    """Load a raw ACS CSV for topic/year and return a slim df with category + estimate."""
    path = files_by_year[topic][year]
    df = pd.read_csv(path)
    est_col = _find_est_col(df)
    cat_col = "Label (Grouping)" if "Label (Grouping)" in df.columns else "Label"
    out = df[[cat_col, est_col]].rename(columns={cat_col:"category", est_col:"estimate"}).copy()
    out["category_n"] = out["category"].map(_norm_label)
    # make numeric; keep original estimate safe
    out["estimate"] = pd.to_numeric(out["estimate"].astype(str).str.replace(",", ""), errors="coerce")
    return out

In [ ]:
def extract_population(year: int) -> pd.DataFrame:
    dfc = _load_raw("population", year)
    total = _pick_any(dfc, ["total:", "total population"])
    return pd.DataFrame([{
        "topic": "population",
        "year": year,
        "population_total": total
    }])

# run for all three years
pop_clean = pd.concat([extract_population(y) for y in [2013, 2018, 2023]], ignore_index=True)
pop_clean

In [ ]:
# standardizes ACS population for the 5-year merge
population_year = (
    pop_clean.rename(columns={"population_total": "population"})[["year","population"]]
             .assign(year=pd.to_numeric(lambda d: d["year"], errors="coerce"),
                     population=pd.to_numeric(lambda d: d["population"], errors="coerce"))
             .dropna(subset=["year","population"])
             .assign(year=lambda d: d["year"].round().astype(int))
             .sort_values("year")
             .reset_index(drop=True)
)
display(population_year)


In [ ]:
def extract_housing(year: int) -> pd.DataFrame:
    # housing units total
    hous = _load_raw("housing", year)
    units = _pick_any(hous, ["total housing units", "total:"])

    # occupied / owner / renter from tenure table
    ten = _load_raw("tenure", year)
    occupied = _pick_any(ten, ["total:"])
    owner    = _pick_any(ten, ["owner occupied", "occupied housing units: owner occupied", "occupied housing units:"])
    renter   = _pick_any(ten, ["renter occupied", "occupied housing units: renter occupied"])

    return pd.DataFrame([{
        "topic": "housing_tenure",
        "year": year,
        "housing_units_total": units,
        "occupied_total": occupied,
        "owner_occupied": owner,
        "renter_occupied": renter
    }])

housing_tenure_clean = pd.concat([extract_housing(y) for y in [2013, 2018, 2023]], ignore_index=True)
housing_tenure_clean

In [ ]:
def extract_vehicles(year: int) -> pd.DataFrame:
    veh = _load_raw("vehicles", year)
    total = _pick_any(veh, ["total:"])
    v0 = _pick_any(veh, ["no vehicle available", "no vehicles available"])
    v1 = _pick_any(veh, ["1 vehicle available"])
    v2 = _pick_any(veh, ["2 vehicles available"])
    v3p = _pick_any(veh, ["3 or more vehicles available", "3 vehicles available"])
    return pd.DataFrame([{
        "topic": "vehicles",
        "year": year,
        "hh_total_for_veh": total,
        "hh_0_veh": v0,
        "hh_1_veh": v1,
        "hh_2_veh": v2,
        "hh_3plus_veh": v3p
    }])

vehicles_clean = pd.concat([extract_vehicles(y) for y in [2013, 2018, 2023]], ignore_index=True)
vehicles_clean

In [ ]:
def extract_commute_time(year: int) -> pd.DataFrame:
    ct = _load_raw("commute_time", year)
    total = _pick_any(ct, ["total:"])
    gt30 = sum([
        _pick_any(ct, "30 to 34 minutes") or 0,
        _pick_any(ct, "35 to 39 minutes") or 0,
        _pick_any(ct, "40 to 44 minutes") or 0,
        _pick_any(ct, "45 to 59 minutes") or 0,
        _pick_any(ct, "60 to 89 minutes") or 0,
        _pick_any(ct, "90 or more minutes") or 0,
    ])
    return pd.DataFrame([{
        "topic": "commute_time",
        "year": year,
        "workers_total": total,
        "gt_30min_commute": gt30
    }])

commute_clean = pd.concat([extract_commute_time(y) for y in [2013, 2018, 2023]], ignore_index=True)
commute_clean

In [ ]:
def extract_transport_type(year: int) -> pd.DataFrame:
    mode = _load_raw("transport_type", year)
    total = _pick_any(mode, ["total:"])
    drive_alone = _pick_any(mode, ["drove alone"])
    carpool = _pick_any(mode, ["carpooled:"])  # header has colon in your files
    public_transport = _pick_any(mode, ["public transportation (excluding taxicab):"])
    walked = _pick_any(mode, ["walked"])
    bicycle = _pick_any(mode, ["bicycle"])
    wfh = _pick_any(mode, ["worked from home"])

    return pd.DataFrame([{
        "topic": "transport_type",
        "year": year,
        "mode_total": total,
        "drive_alone": drive_alone,
        "carpool": carpool,
        "public_transport": public_transport,
        "walked": walked,
        "bicycle": bicycle,
        "worked_from_home": wfh
    }])

transport_clean = pd.concat([extract_transport_type(y) for y in [2013, 2018, 2023]], ignore_index=True)
transport_clean

In [ ]:
def show_mode_labels(year):
    mode = _load_raw("transport_type", year)
    print(f"\n--- transport_type labels {year} ---")
    for s in sorted(mode["category"].dropna().unique())[:200]:
        print(s)

show_mode_labels(2013)
show_mode_labels(2018)

In [ ]:
def extract_transport_type(year: int) -> pd.DataFrame:
    mode = _load_raw("transport_type", year)

    total             = _pick_any(mode, ["total:"])
    drive_alone       = _pick_any(mode, ["drove alone"])
    carpool           = _pick_any(mode, ["carpooled:", "carpooled"])  # with/without colon
    public_transport  = _pick_any(mode, [
        "public transportation (excluding taxicab):",
        "public transportation (excluding taxicab)"
    ])
    walked            = _pick_any(mode, ["walked"])
    bicycle           = _pick_any(mode, ["bicycle"])

    # Handle pre-2020 wording and missing values:
    wfh = _pick_any(mode, ["worked from home", "worked at home", "work at home"])
    # If still None for earlier years, treat as 0 rather than NaN
    if wfh is None:
        wfh = 0.0

    # Also default other missing subcategories to 0 so charts don’t break
    for var in ("drive_alone","carpool","public_transport","walked","bicycle"):
        if locals()[var] is None:
            locals()[var] = 0.0

    return pd.DataFrame([{
        "topic": "transport_type",
        "year": year,
        "mode_total": total,
        "drive_alone": drive_alone,
        "carpool": carpool,
        "public_transport": public_transport,
        "walked": walked,
        "bicycle": bicycle,
        "worked_from_home": wfh
    }])

# re-run for all three years
transport_clean = pd.concat([extract_transport_type(y) for y in [2013, 2018, 2023]], ignore_index=True)
display(transport_clean)

In [ ]:
chk = transport_clean.copy()
chk["sub_sum"] = chk[["drive_alone","carpool","public_transport","walked","bicycle","worked_from_home"]].sum(axis=1)
print(chk[["year","mode_total","sub_sum"]])

In [ ]:
# folders
ROOT = Path(".")
FIG_DIR = ROOT / "figures"
OUT_DIR = ROOT / "outputs"
FIG_DIR.mkdir(exist_ok=True)
OUT_DIR.mkdir(exist_ok=True)

# clean default style
sns.set_context("talk")
sns.set_style("whitegrid")

In [ ]:
ROOT = Path(".")
FIG_DIR = ROOT / "figures"
OUT_DIR = ROOT / "outputs"
FIG_DIR.mkdir(exist_ok=True)
OUT_DIR.mkdir(exist_ok=True)

sns.set_context("talk")
sns.set_style("whitegrid")

In [ ]:
# discovers a previously exported tidy file (common locations)
CANDIDATE_TIDY = [
    Path("aadt_davidson_long.csv"),
    Path("outputs/aadt_davidson_long.csv"),
    Path("out/aadt_davidson_long.csv"),
]
TRAFFIC_TIDY_CSV = next((p for p in CANDIDATE_TIDY if p.exists()), None)

# discovers a raw TDOT file by pattern if tidy is missing
TRAFFIC_RAW_CSV = None if TRAFFIC_TIDY_CSV else next(iter(sorted(Path(".").glob("Traffic_Lines_*.csv"))), None)

def _guess_year_columns(df: pd.DataFrame) -> list[str]:
    """returns columns whose names contain a 4-digit year"""
    cols = []
    for c in df.columns:
        if re.search(r"(19|20)\d{2}", str(c)):
            cols.append(c)
    # preserves original order and uniqueness
    seen, out = set(), []
    for c in cols:
        if c not in seen:
            out.append(c); seen.add(c)
    return out


In [ ]:
def _detect_long_columns(df: pd.DataFrame) -> tuple[str | None, str | None]:
    """returns (year_col, aadt_col) when data already stores one row per year"""
    lower = {c: str(c).lower() for c in df.columns}
    # finds a year column by name or by values
    year_col = next((c for c, s in lower.items() if s == "year" or "year" in s), None)
    if year_col is None:
        # detects numeric-like year columns by values
        for c in df.columns:
            s = pd.to_numeric(df[c], errors="coerce")
            if s.notna().any():
                vals = s.dropna()
                if (vals.between(1990, 2035).mean() > 0.5):  # majority looks like years
                    year_col = c
                    break
    # finds an AADT-like column
    aadt_col = next((c for c, s in lower.items() if "aadt" in s), None)
    if aadt_col is None:
        # falls back to a volume/count-style numeric column
        candidates = [c for c, s in lower.items() if any(k in s for k in ["vol", "count", "aadt", "traffic"])]
        candidates = [c for c in candidates if pd.api.types.is_numeric_dtype(df[c]) or pd.to_numeric(df[c], errors="coerce").notna().mean() > 0.8]
        aadt_col = candidates[0] if candidates else None
    return year_col, aadt_col

In [ ]:
def _tidy_from_wide(df: pd.DataFrame, id_col: str, keep_cols: tuple[str, ...] = ()) -> pd.DataFrame:
    """melts wide year columns into long format and extracts numeric year"""
    year_cols = _guess_year_columns(df)
    if not year_cols:
        raise ValueError("No year-like columns found in traffic raw file.")
    out = df[list(keep_cols) + [id_col] + year_cols].melt(
        id_vars=[id_col] + list(keep_cols),
        value_vars=year_cols,
        var_name="year_col",
        value_name="aadt"
    )
    out["year"] = pd.to_numeric(out["year_col"].astype(str).str.extract(r"((?:19|20)\d{2})")[0], errors="coerce")
    out = out.drop(columns=["year_col"])
    return out

In [ ]:
if TRAFFIC_TIDY_CSV and TRAFFIC_TIDY_CSV.exists():
    traffic_long = pd.read_csv(TRAFFIC_TIDY_CSV)

    # standardizes expected columns
    if "year" not in traffic_long.columns:
        yc = [c for c in traffic_long.columns if re.search(r"(19|20)\d{2}", str(c))]
        if yc:
            traffic_long = traffic_long.rename(columns={yc[0]: "year"})
    if "aadt" not in traffic_long.columns:
        num_cols = traffic_long.select_dtypes(include=[np.number]).columns.tolist()
        num_cols = [c for c in num_cols if c != "year"]
        if num_cols:
            traffic_long = traffic_long.rename(columns={num_cols[0]: "aadt"})

else:
    if TRAFFIC_RAW_CSV is None:
        raise FileNotFoundError("No tidy AADT file found and no raw 'Traffic_Lines_*.csv' present.")

    traffic_raw = pd.read_csv(TRAFFIC_RAW_CSV, low_memory=False)

    # detects format
    year_col, aadt_col = _detect_long_columns(traffic_raw)
    wide_year_cols = _guess_year_columns(traffic_raw)

    if year_col and aadt_col:
        # selects columns by index to avoid duplicate-name DataFrame slices
        def _first_index(df: pd.DataFrame, label) -> int | None:
            matches = [i for i, c in enumerate(df.columns) if str(c) == str(label)]
            return matches[0] if matches else None

        yi = _first_index(traffic_raw, year_col)
        ai = _first_index(traffic_raw, aadt_col)
        if yi is None or ai is None:
            raise ValueError(f"Could not resolve indices for year='{year_col}' and aadt='{aadt_col}'.")

        # converts selected columns to numeric Series
        year_s = pd.to_numeric(traffic_raw.iloc[:, yi], errors="coerce")
        aadt_s = pd.to_numeric(traffic_raw.iloc[:, ai], errors="coerce")

        # builds long-format frame and removes missing values
        traffic_long = (
            pd.DataFrame({"year": year_s, "aadt": aadt_s})
            .dropna(subset=["year", "aadt"])
        )

    elif wide_year_cols:
        # treats file as wide; selects an id column when present
        candidate_ids = [c for c in ["STATION","STATION_ID","STATIONID","SEGMENT_ID","SEGMENTID","GISID","OBJECTID","ID"] if c in traffic_raw.columns]
        id_col = candidate_ids[0] if candidate_ids else traffic_raw.columns[0]
        traffic_long = _tidy_from_wide(traffic_raw, id_col=id_col, keep_cols=())
        traffic_long = traffic_long.rename(columns={id_col: "segment_id"})

    else:
        # raises with diagnostics to show inspected columns
        raise ValueError(
            "Traffic file does not expose year-like columns in headers and does not contain a clear YEAR/AADT pair.\n"
            f"Columns seen: {list(traffic_raw.columns)[:25]}..."
        )

# aggregates to year level
traffic_year = (
    traffic_long
    .dropna(subset=["year", "aadt"])
    .groupby("year", as_index=False)["aadt"].sum()
    .rename(columns={"aadt": "aadt_total"})
    .sort_values("year")
)


In [ ]:
# standardizes traffic to have ['year','aadt_total']
if "aadt_total" not in traffic_year.columns:
    if "AADT" in traffic_year.columns:
        traffic_year_std = (
            traffic_year.groupby("year", as_index=False)["AADT"].sum()
                        .rename(columns={"AADT": "aadt_total"})
        )
    else:
        raise ValueError("traffic_year must have 'aadt_total' or 'AADT' column.")
else:
    traffic_year_std = traffic_year.copy()

traffic_year_fix = (
    traffic_year_std.assign(
        year=pd.to_numeric(traffic_year_std["year"], errors="coerce"),
        aadt_total=pd.to_numeric(traffic_year_std["aadt_total"], errors="coerce"),
    )
    .dropna(subset=["year", "aadt_total"])
    .assign(year=lambda d: d["year"].round().astype(int))
    .query("1990 <= year <= 2035")
    .drop_duplicates(subset=["year"], keep="last")
    .sort_values("year")[["year", "aadt_total"]]
)

# standardizes population to have ['year','population']
if "population" not in population_year.columns:
    if "population_total" in population_year.columns:
        population_year_std = population_year.rename(columns={"population_total": "population"})
    else:
        raise ValueError("population_year must have 'population' (or 'population_total') column.")
else:
    population_year_std = population_year.copy()

population_year_fix = (
    population_year_std.assign(
        year=pd.to_numeric(population_year_std["year"], errors="coerce"),
        population=pd.to_numeric(population_year_std["population"], errors="coerce"),
    )
    .dropna(subset=["year", "population"])
    .assign(year=lambda d: d["year"].round().astype(int))
    .query("1990 <= year <= 2035")
    .drop_duplicates(subset=["year"], keep="last")
    .sort_values("year")[["year", "population"]]
)

# builds year-indexed traffic frame and computes strict 5-year rolling mean
ty = traffic_year_fix.set_index("year").sort_index()
ty["aadt_total_5y_mean"] = ty["aadt_total"].rolling(window=5, min_periods=5).mean()

# prepares merge keys (ACS uses end-year label)
traffic_5y = (
    ty.reset_index()
      .rename(columns={"year": "end_year"})
      .loc[:, ["end_year", "aadt_total_5y_mean"]]
)
pop_5y = population_year_fix.rename(columns={"year": "end_year"})

# merges and computes per-capita 5-year metric
df5 = (
    pop_5y.merge(traffic_5y, on="end_year", how="left")
          .assign(
              period=lambda d: d["end_year"].apply(lambda y: f"{y-4}–{y}"),
              aadt_per_capita_5y=lambda d: d["aadt_total_5y_mean"] / d["population"],
          )
          .dropna(subset=["aadt_total_5y_mean", "population"])
          .sort_values("end_year")
          .reset_index(drop=True)
)

print(
    "traffic years:", traffic_year_fix["year"].min(), "→", traffic_year_fix["year"].max(),
    "| population ACS end-years:", population_year_fix["year"].tolist(),
    "| rows after 5-year alignment:", len(df5)
)
display(df5.head(10))

In [ ]:
# reads the year totals produced in Data Check and standardizes to traffic_year
TRAFFIC_TOTALS_CSV = Path("outputs/aadt_davidson_year_totals.csv")

traffic_year = pd.read_csv(TRAFFIC_TOTALS_CSV)

# normalizes column names just in case
if "AADT_YEAR" in traffic_year.columns and "year" not in traffic_year.columns:
    traffic_year = traffic_year.rename(columns={"AADT_YEAR": "year"})
if "aadt_total" not in traffic_year.columns and "AADT" in traffic_year.columns:
    traffic_year = (traffic_year.groupby("year", as_index=False)["AADT"]
                               .sum()
                               .rename(columns={"AADT": "aadt_total"}))

# coerces types and sorts
traffic_year["year"] = pd.to_numeric(traffic_year["year"], errors="coerce")
traffic_year["aadt_total"] = pd.to_numeric(traffic_year["aadt_total"], errors="coerce")
traffic_year = (traffic_year.dropna(subset=["year","aadt_total"])
                             .assign(year=lambda d: d["year"].round().astype(int))
                             .sort_values("year")
                             .reset_index(drop=True))

display(traffic_year)

YEAR_VALUE_RE = re.compile(r"(?:19|20)\d{2}")  # matches a 4-digit year without capture groups

def _first_estimate_column(df: pd.DataFrame) -> str:
    """returns the first column that ends with '!!Estimate' or contains 'Estimate'"""
    for c in df.columns:
        if isinstance(c, str) and c.endswith("!!Estimate"):
            return c
    for c in df.columns:
        if isinstance(c, str) and "Estimate" in c:
            return c
    raise ValueError("Estimate column not found (expects something like '...!!Estimate').")

def _to_number(x):
    """converts strings with commas/± to numeric"""
    return pd.to_numeric(str(x).replace(",", "").replace("±", "").strip(), errors="coerce")

def _load_population_years(root: Path = Path(".")) -> pd.DataFrame:
    """loads USCPOPDATA CSVs, extracts year and population, and aggregates to year"""
    files = sorted(root.glob("USCPOPDATA*.csv"))
    if not files and (root / "USCPOPDATA.csv").exists():
        files = [root / "USCPOPDATA.csv"]
    if not files:
        raise FileNotFoundError("No USCPOPDATA CSV found.")

    frames = []
    for p in files:
        df = pd.read_csv(p)

        # identifies a year column by name or by values (uses non-capturing regex to avoid warnings)
        year_col = next((c for c in df.columns if str(c).strip().lower() in {"year", "yr", "time"}), None)
        if year_col is None:
            for c in df.columns:
                if df[c].astype(str).str.contains(YEAR_VALUE_RE, na=False).any():
                    year_col = c
                    break

        est_col = _first_estimate_column(df)

        # extracts year values from the detected column or, if absent, from the filename
        if year_col is not None:
            extracted_year = df[year_col].astype(str).str.extract(YEAR_VALUE_RE, expand=False)
        else:
            m = YEAR_VALUE_RE.search(p.name)
            const_year = m.group(0) if m else np.nan
            extracted_year = pd.Series([const_year] * len(df), index=df.index)

        tmp = (
            df.assign(
                year=pd.to_numeric(extracted_year, errors="coerce"),
                population=lambda d: d[est_col].map(_to_number)
            )
            .dropna(subset=["year", "population"])
            .groupby("year", as_index=False)["population"].sum()
        )
        frames.append(tmp)

    population_year = (
        pd.concat(frames, ignore_index=True)
        .groupby("year", as_index=False)["population"].sum()
        .sort_values("year")
    )
    return population_year

population_year = _load_population_years()

# standardizes traffic years and removes duplicates
traffic_year_fix = (
    traffic_year.assign(year=pd.to_numeric(traffic_year["year"], errors="coerce"))
                .dropna(subset=["year"])
                .assign(year=lambda d: d["year"].round().astype(int))
                .query("1990 <= year <= 2035")
                .drop_duplicates(subset=["year"], keep="last")
                .sort_values("year")
)

# standardizes population years (ACS 5-year end-years)
population_year_fix = (
    population_year.assign(year=pd.to_numeric(population_year["year"], errors="coerce"))
                   .dropna(subset=["year"])
                   .assign(year=lambda d: d["year"].round().astype(int))
                   .query("1990 <= year <= 2035")
                   .drop_duplicates(subset=["year"], keep="last")
                   .sort_values("year")
)

# computes traffic 5-year rolling mean ending at each year (uses >=3 years when edges are sparse)
ty = traffic_year_fix.set_index("year").sort_index()
ty["aadt_total_5y_mean"] = (
    ty["aadt_total"].rolling(window=5, min_periods=3).mean()
)

traffic_5y = (
    ty.reset_index()
      .rename(columns={"year": "end_year"})
      [["end_year", "aadt_total_5y_mean"]]
)

# prepares ACS 5-year table keyed by end_year
pop_5y = population_year_fix.rename(columns={"year": "end_year"})

# Merges ACS 5-year population with traffic 5-year rolling mean ===
df5 = (
    pop_5y.merge(traffic_5y, on="end_year", how="left")
          .dropna(subset=["aadt_total_5y_mean"])
          .assign(
              period=lambda d: d["end_year"].apply(lambda y: f"{y-4}–{y}"),
              aadt_per_capita_5y=lambda d: d["aadt_total_5y_mean"] / d["population"]
          )
          .sort_values("end_year")
          .reset_index(drop=True)
)

print("Rows after 5-year alignment:", len(df5))
display(df5.head())

traffic_year_fix = (
    traffic_year.assign(year=pd.to_numeric(traffic_year["year"], errors="coerce"),
                        aadt_total=pd.to_numeric(traffic_year.get("aadt_total", traffic_year.get("AADT")), errors="coerce"))
                .dropna(subset=["year","aadt_total"])
                .assign(year=lambda d: d["year"].round().astype(int))
                .query("1990 <= year <= 2035")
                .drop_duplicates(subset=["year"], keep="last")
                .sort_values("year")[["year","aadt_total"]]
)

population_year_fix = (
    population_year.assign(year=pd.to_numeric(population_year["year"], errors="coerce"),
                           population=pd.to_numeric(population_year.get("population"), errors="coerce"))
                   .dropna(subset=["year","population"])
                   .assign(year=lambda d: d["year"].round().astype(int))
                   .query("1990 <= year <= 2035")
                   .drop_duplicates(subset=["year"], keep="last")
                   .sort_values("year")[["year","population"]]
)

ty = traffic_year_fix.set_index("year").sort_index()
ty["aadt_total_5y_mean"] = ty["aadt_total"].rolling(window=5, min_periods=5).mean()

traffic_5y = ty.reset_index().rename(columns={"year":"end_year"})[["end_year","aadt_total_5y_mean"]]
pop_5y = population_year_fix.rename(columns={"year":"end_year"})

df5 = (
    pop_5y.merge(traffic_5y, on="end_year", how="left")
          .assign(period=lambda d: d["end_year"].apply(lambda y: f"{y-4}–{y}"),
                  aadt_per_capita_5y=lambda d: d["aadt_total_5y_mean"] / d["population"])
          .dropna(subset=["aadt_total_5y_mean","population"])
          .sort_values("end_year")
          .reset_index(drop=True)
)

# Exports (ACS 5-year aligned): writes merged CSV and saves figures ===
ROOT = Path(".")
FIG_DIR = ROOT / "figures"
OUT_DIR = ROOT / "outputs"
FIG_DIR.mkdir(exist_ok=True)
OUT_DIR.mkdir(exist_ok=True)

# writes merged 5-year table
out_csv_5y = OUT_DIR / "traffic_population_merged_5y.csv"
df5.to_csv(out_csv_5y, index=False)

# uses period labels for tick marks
x = df5["end_year"]
xticks = df5["period"]

# population trend (ACS 5-year, labeled by period)
plt.figure(figsize=(10, 4))
plt.plot(x, df5["population"], marker="o")
plt.title("Population Trend (ACS 5-year)")
plt.xlabel("ACS Period End Year")
plt.ylabel("Population")
plt.xticks(x, xticks, rotation=0)
plt.tight_layout()
plt.savefig(FIG_DIR / "population_trend_5y.png", dpi=300, bbox_inches="tight")
plt.close()

# traffic trend (5-year rolling mean, labeled by period)
plt.figure(figsize=(10, 4))
plt.plot(x, df5["aadt_total_5y_mean"], marker="o")
plt.title("Total AADT (5-year rolling mean)")
plt.xlabel("ACS Period End Year")
plt.ylabel("Total AADT (5y mean)")
plt.xticks(x, xticks, rotation=0)
plt.tight_layout()
plt.savefig(FIG_DIR / "traffic_trend_5y.png", dpi=300, bbox_inches="tight")
plt.close()

# population vs. traffic (ACS 5-year vs 5-year mean AADT)
plt.figure(figsize=(5, 5))
plt.scatter(df5["population"], df5["aadt_total_5y_mean"])
plt.title("Traffic vs. Population (5-year aligned)")
plt.xlabel("Population (ACS 5-year)")
plt.ylabel("Total AADT (5y mean)")
plt.tight_layout()
plt.savefig(FIG_DIR / "pop_vs_traffic_scatter_5y.png", dpi=300, bbox_inches="tight")
plt.close()

# per-capita AADT (5-year aligned)
plt.figure(figsize=(10, 4))
plt.plot(x, df5["aadt_per_capita_5y"], marker="o")
plt.title("Per-Capita Traffic (AADT per person, 5-year aligned)")
plt.xlabel("ACS Period End Year")
plt.ylabel("AADT / Capita (5y mean)")
plt.xticks(x, xticks, rotation=0)
plt.tight_layout()
plt.savefig(FIG_DIR / "per_capita_congestion_5y.png", dpi=300, bbox_inches="tight")
plt.close()

print("traffic_year shape:", traffic_year.shape)
print(traffic_year.head(3)); print(traffic_year.tail(3))

print("\npopulation_year shape:", population_year.shape)
print(population_year.head(3)); print(population_year.tail(3))

years_traffic = pd.to_numeric(traffic_year["year"], errors="coerce").dropna().astype(int)
years_pop     = pd.to_numeric(population_year["year"], errors="coerce").dropna().astype(int)
print("\ntraffic years:", sorted(years_traffic.unique())[:20], "...", sorted(years_traffic.unique())[-5:])
print("population years (ACS end-years):", sorted(years_pop.unique()))
print("exact overlap:", sorted(set(years_traffic) & set(years_pop)))

In [ ]:
traffic_year_fix = (
    traffic_year.assign(year=pd.to_numeric(traffic_year["year"], errors="coerce"))
                .dropna(subset=["year"])
                .assign(year=lambda d: d["year"].round().astype(int))
                .query("1990 <= year <= 2035")
                .drop_duplicates(subset=["year"], keep="last")
                .sort_values("year")
)

population_year_fix = (
    population_year.assign(year=pd.to_numeric(population_year["year"], errors="coerce"))
                   .dropna(subset=["year"])
                   .assign(year=lambda d: d["year"].round().astype(int))
                   .query("1990 <= year <= 2035")
                   .drop_duplicates(subset=["year"], keep="last")
                   .sort_values("year")
)

# computes 5-year rolling mean for traffic 
ty["aadt_total_5y_mean"] = ty["aadt_total"].rolling(window=5, min_periods=5).mean()

traffic_5y = (
    ty.reset_index()
      .rename(columns={"year": "end_year"})
      [["end_year", "aadt_total_5y_mean"]]
)

pop_5y = population_year_fix.rename(columns={"year": "end_year"})

df5 = (
    pop_5y.merge(traffic_5y, on="end_year", how="left")
          .assign(
              period=lambda d: d["end_year"].apply(lambda y: f"{y-4}–{y}"),
              aadt_per_capita_5y=lambda d: d["aadt_total_5y_mean"] / d["population"]
          )
          .dropna(subset=["aadt_total_5y_mean", "population"])
          .sort_values("end_year")
          .reset_index(drop=True)
)

print("rows after 5-year alignment:", len(df5))
display(df5.head())

In [ ]:
ROOT = Path(".")
FIG_DIR = ROOT / "figures"
OUT_DIR = ROOT / "outputs"
FIG_DIR.mkdir(exist_ok=True)
OUT_DIR.mkdir(exist_ok=True)

if len(df5) == 0:
    raise ValueError("No rows after 5-year alignment. Check the diagnostics printout to align years/columns.")

# writes merged 5-year table
(df5[["end_year","period","population","aadt_total_5y_mean","aadt_per_capita_5y"]]
 .to_csv(OUT_DIR / "traffic_population_merged_5y.csv", index=False))

x = df5["end_year"]; xticks = df5["period"]

plt.figure(figsize=(10,4))
plt.plot(x, df5["population"], marker="o")
plt.title("Population Trend (ACS 5-year)")
plt.xlabel("ACS Period End Year"); plt.ylabel("Population")
plt.xticks(x, xticks, rotation=0)
plt.tight_layout(); plt.savefig(FIG_DIR/"population_trend_5y.png", dpi=300, bbox_inches="tight"); plt.close()

plt.figure(figsize=(10,4))
plt.plot(x, df5["aadt_total_5y_mean"], marker="o")
plt.title("Total AADT (5-year rolling mean)")
plt.xlabel("ACS Period End Year"); plt.ylabel("Total AADT (5y mean)")
plt.xticks(x, xticks, rotation=0)
plt.tight_layout(); plt.savefig(FIG_DIR/"traffic_trend_5y.png", dpi=300, bbox_inches="tight"); plt.close()

plt.figure(figsize=(5,5))
plt.scatter(df5["population"], df5["aadt_total_5y_mean"])
plt.title("Traffic vs. Population (5-year aligned)")
plt.xlabel("Population (ACS 5-year)"); plt.ylabel("Total AADT (5y mean)")
plt.tight_layout(); plt.savefig(FIG_DIR/"pop_vs_traffic_scatter_5y.png", dpi=300, bbox_inches="tight"); plt.close()

plt.figure(figsize=(10,4))
plt.plot(x, df5["aadt_per_capita_5y"], marker="o")
plt.title("Per-Capita Traffic (AADT per person, 5-year aligned)")
plt.xlabel("ACS Period End Year"); plt.ylabel("AADT / Capita (5y mean)")
plt.xticks(x, xticks, rotation=0)
plt.tight_layout(); plt.savefig(FIG_DIR/"per_capita_congestion_5y.png", dpi=300, bbox_inches="tight"); plt.close()